In [1]:
import torch
from collections import defaultdict
import matplotlib.pyplot as plt
import time


from rendering import *
from nerf import *

import metrics

from nerf_dataloader import HololensSimpleDataset

torch.backends.cudnn.benchmark = True

dataset = HololensSimpleDataset('../Hololens2-Capture/datanerf/',100)

In [3]:
embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_coarse = NeRF()
nerf_fine = NeRF()

nerf_coarse.cuda().eval()
nerf_fine.cuda().eval();

In [14]:
models = [nerf_coarse, nerf_fine]
embeddings = [embedding_xyz, embedding_dir]

N_samples = 64
N_importance = 64
use_disp = False
chunk = 1024*32*4

img_wh = (720,1280)

def f(rays):
    """Do batched inference on rays using chunk."""
    if len(rays.shape) == 1:
        rays = rays.reshape(1,*rays.shape)
    B = rays.shape[0]
    results = defaultdict(list)
    for i in range(0, B, chunk):
        rendered_ray_chunks = \
            render_rays(models,
                        embeddings,
                        rays[i:i+chunk],
                        N_samples,
                        use_disp,
                        0,
                        0,
                        N_importance,
                        chunk,
                        dataset.white_back,
                        test_time=True)

        for k, v in rendered_ray_chunks.items():
            results[k] += [v]

    for k, v in results.items():
        results[k] = torch.cat(v, 0)
    return results

In [15]:
sample = dataset[0]
rays = sample['rays'].cuda()

t = time.time()
results = f(rays)
torch.cuda.synchronize()
print(time.time()-t)

0.3946404457092285
